#GANSpace - Discovering Interpretable GAN Controls 

Using https://github.com/harskish/ganspace to find latent directions in a StyleGAN2 model.

Notebook put together by [@realmeatyhuman](https://twitter.com/realmeatyhuman)



In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
#@title Mount Google Drive (Optional)
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/



## Setup
Hit play on all the cells below, and everything should run smoothly. The install takes around half a minute.



In [ ]:
# Clone git
!git clone https://github.com/KV9801/ganspace
%cd ganspace

Cloning into 'ganspace'...
remote: Enumerating objects: 320, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 320 (delta 31), reused 22 (delta 22), pack-reused 273
Receiving objects: 100% (320/320), 46.78 MiB | 24.72 MiB/s, done.
Resolving deltas: 100% (101/101), done.
/content/ganspace


In [ ]:
#@title Install remaining packages
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'''))
!pip install fbpca boto3
!git submodule update --init --recursive
!python -c "import nltk; nltk.download('wordnet')"

# Custom OPs no longer required
#!pip install Ninja
#%cd models/stylegan2/stylegan2-pytorch/op
#!python setup.py install
#!python -c "import torch; import upfirdn2d_op; import fused; print('OK')"
#%cd "/content/ganspace"

<IPython.core.display.Javascript object>

     |████████████████████████████████| 131 kB 12.7 MB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 8.4 MB 42.8 MB/s 
     |████████████████████████████████| 138 kB 37.6 MB/s 
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11376 sha256=c7a584bc4867b218fb392935078e6a5b20a7126391776cdb144597128377ad96
  Stored in directory: /root/.cache/pip/wheels/93/08/0c/1b9866c35c8d3f136d100dfe88036a32e0795437daca089f70
Successfully built fbpca
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
kapre 0.3.6 requires tensorflow>=2.0.0, b

Submodule 'stylegan/stylegan_tf' (https://github.com/NVlabs/stylegan.git) registered for path 'models/stylegan/stylegan_tf'
Submodule 'stylegan2/stylegan2-pytorch' (https://github.com/harskish/stylegan2-pytorch.git) registered for path 'models/stylegan2/stylegan2-pytorch'
Cloning into '/content/ganspace/models/stylegan/stylegan_tf'...
Cloning into '/content/ganspace/models/stylegan2/stylegan2-pytorch'...
Submodule path 'models/stylegan/stylegan_tf': checked out '66813a32aac5045fcde72751522a0c0ba963f6f2'
Submodule path 'models/stylegan2/stylegan2-pytorch': checked out '91ea2a7a4320701535466cce942c9e099d65670e'
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## Download pre-trained weights

In [ ]:
# Download converted pretrained StyleGAN2-ADA model (pizza10)
!gdown --id 1Chk0SSmDKaUPgeG3Inu1vMLUAWla-wYS

Downloading...
From: https://drive.google.com/uc?id=1Chk0SSmDKaUPgeG3Inu1vMLUAWla-wYS
To: /content/network-snapshot-010483.pt
100% 99.8M/99.8M [00:01<00:00, 75.8MB/s]


# Run PCA Analysis

##Options


```
Command line paramaters:
  --model      one of [ProGAN, BigGAN-512, BigGAN-256, BigGAN-128, StyleGAN, StyleGAN2]
  --class      class name; leave empty to list options
  --layer      layer at which to perform PCA; leave empty to list options
  --use_w      treat W as the main latent space (StyleGAN / StyleGAN2)
  --inputs     load previously exported edits from directory
  --sigma      number of stdevs to use in visualize.py
  -n           number of PCA samples
  -b           override automatic minibatch size detection
  -c           number of components to keep

```




In [ ]:
%cd ganspace/

/content/ganspace


In [ ]:
model_name = 'StyleGAN2' 
model_class = 'pizza'
num_components = 512 # c
num_pca = 1         # n
batch_size = 1

In [ ]:
#Check layers available for analysis by passing dummy name
!python visualize.py --model $model_name --class $model_class --use_w --layer=dummy_name

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.
[09.12 22:25] StyleGAN2, dummy_name, ipca
Layer 'dummy_name' not found in model!
Available layers: 
style
style.0
style.1
style.2
style.3
style.4
style.5
style.6
style.7
style.8
input
conv1
conv1.conv
conv1.conv.modulation
conv1.noise
conv1.activate
to_rgb1
to_rgb1.conv
to_rgb1.conv.modulation
convs
convs.0
convs.0.conv
convs.0.conv.blur
convs.0.conv.modulation
convs.0.noise
convs.0.activate
convs.1
convs.1.conv
convs.1.conv.modulation
convs.1.noise
convs.1.activate
convs.2
convs.2.conv
convs.2.conv.blur
convs.2.conv.modulation
convs.2.noise
convs.2.activate
convs.3
convs.3.conv
convs.3.conv.modulation
convs.3.noise
convs.3.ac

Add chosen layer in as --layer argument:

In [ ]:
!python visualize.py --model $model_name --class $model_class --use_w --layer=style -c $num_components -n $num_pca -b $batch_size

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.
[12.12 20:54] StyleGAN2, style, ipca
Feature shape: torch.Size([1, 512])
Not cached
[12.12 20:54] Computing stylegan2-pizza_style_ipca_c512_n1_w.npz
Reusing InstrumentedModel instance
Using W latent space
Feature shape: torch.Size([1, 512])
B=1, N=1, dims=512, N/dims=0.0
Sampling latents: 100% 2001/2001 [00:03<00:00, 526.58it/s]
Fitting batches (NB=2000): 100% 1/1 [00:00<00:00,  2.84it/s]
Total time: 0:00:04.593769
Batch size: 1
[12.12 20:54] Creating visualizations
Sparsity: 0.00
<Figure size 1400x1200 with 14 Axes>
<Figure size 1400x1200 with 14 Axes>
Random images:   0% 0/10 [00:00<?, ?it/s]<Figure size 1400x1200 with 14 Ax

In [ ]:
# !python visualize.py --model=$model_name --class=$model_class --use_w --layer=style -b=1000

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.
[09.12 22:31] StyleGAN2, style, ipca
Feature shape: torch.Size([1, 512])
Batch size: 10000
[09.12 22:31] Creating visualizations
Sparsity: 0.00
<Figure size 1400x1200 with 14 Axes>
<Figure size 1400x1200 with 14 Axes>
Random images:   0% 0/10 [00:00<?, ?it/s]<Figure size 1400x1200 with 14 Axes>
Random images:  10% 1/10 [00:06<01:02,  6.99s/it]<Figure size 1400x1200 with 14 Axes>
Random images:  20% 2/10 [00:14<00:56,  7.03s/it]<Figure size 1400x1200 with 14 Axes>
Random images:  30% 3/10 [00:21<00:50,  7.23s/it]<Figure size 1400x1200 with 14 Axes>
Random images:  40% 4/10 [00:28<00:43,  7.28s/it]<Figure size 1400x1200 with 14 

In [ ]:
!zip -r samples1.zip "/content/ganspace/out/StyleGAN2-pizza" #zip up samples for download

  adding: content/ganspace/out/StyleGAN2-pizza/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ipca/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ipca/inst/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ipca/comp/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ipca/summ/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ipca/summ/samp4_real_W.jpg (deflated 3%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ipca/summ/samp2_real_W.jpg (deflated 3%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ipca/summ/samp1_real_W.jpg (deflated 3%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ipca/summ/samp7_real_W.jpg (deflated 2%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ipca/summ/samp0_real_W.jpg (deflated 3%)
  adding: content/ganspace/out/StyleGAN2-pizza/style/ipca/summ/components_W.jpg (deflated 2%)
  adding: c

In [ ]:
%cp -r "/content/ganspace/out/StyleGAN2-pizza/style/ipca/summ" "/content/drive/My Drive/stylegan2-pizza10/pca_n1_c512"

In [ ]:
%cp -r "/content/ganspace/cache/components" "/content/drive/My Drive/stylegan2-pizza10/gen" #copying components over to google drive

# Explore Directions!

After running visualize.py, your components will be stored in an npz file in /content/ganspace/cache/components/ - below the npz file is unpacked, and a component/direction is chosen at random. 

Using the UI, you can explore the latent direction and give it a name, which will be appeneded to the named_directions dictionary and saved as 'direction_name.npy' for later use.


In [ ]:
# Load model
from IPython.utils import io
import torch
import PIL
import numpy as np
import ipywidgets as widgets
from PIL import Image
import imageio
from models import get_instrumented_model
from decomposition import get_or_compute
from config import Config
from skimage import img_as_ubyte

# Speed up computation
torch.autograd.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

# Specify model to use
config = Config(
  model='StyleGAN2',
  layer='style',
  output_class='pizza',
  components=80,
  use_w=True,
  batch_size=5_000, # style layer quite small
)

inst = get_instrumented_model(config.model, config.output_class,
                              config.layer, torch.device('cuda'), use_w=config.use_w)

path_to_components = get_or_compute(config, inst)

model = inst.model

named_directions = {} #init named_directions dict to save directions

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.
Not cached
[12.12 20:52] Computing stylegan2-pizza_style_ipca_c80_n300000_w.npz
Reusing InstrumentedModel instance
Using W latent space
Feature shape: torch.Size([1, 512])
B=5000, N=300000, dims=512, N/dims=585.9


Fitting batches (NB=5000): 100%|##########| 60/60 [00:40<00:00,  1.49it/s]


Total time: 0:00:49.424638


In [ ]:
#@title Load a component at random

comps = np.load(path_to_components)
lst = comps.files
latent_dirs = []
latent_stdevs = []

load_activations = False

for item in lst:
    if load_activations:
      if item == 'act_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'act_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
    else:
      if item == 'lat_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'lat_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
            
#load one at random 
num = np.random.randint(20)
if num in named_directions.values():
  print(f'Direction already named: {list(named_directions.keys())[list(named_directions.values()).index(num)]}')

random_dir = latent_dirs[num]
random_dir_stdev = latent_stdevs[num]

print(f'Loaded Component No. {num}')


Loaded Component No. 6


In [ ]:
#@title Run UI (save component with Enter key)
from ipywidgets import fixed

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def name_direction(sender):
  if not text.value:
    print('Please name the direction before saving')
    return
    
  if num in named_directions.values():
    target_key = list(named_directions.keys())[list(named_directions.values()).index(num)]
    print(f'Direction already named: {target_key}')
    print(f'Overwriting... ')
    del(named_directions[target_key])
  named_directions[text.value] = [num, start_layer.value, end_layer.value]
  save_direction(random_dir, text.value)
  for item in named_directions:
    print(item, named_directions[item])

def save_direction(direction, filename):
  filename += ".npy"
  np.save(filename, direction, allow_pickle=True, fix_imports=True)
  print(f'Latent direction saved as {filename}')

def display_sample_pytorch(seed, truncation, direction, distance, scale, start, end, disp=True, save=None, noise_spec=None):
    # blockPrint()
    with io.capture_output() as captured:
      w = model.sample_latent(1, seed=seed).cpu().numpy()

      model.truncation = truncation
      w = [w]*model.get_max_latents() # one per layer
      for l in range(start, end):
          w[l] = w[l] + direction * distance * scale

      #save image and display
      out = model.sample_np(w)
      final_im = Image.fromarray((out * 255).astype(np.uint8)).resize((500,500),Image.LANCZOS)

    if disp:
      display(final_im)
    if save is not None:
      if disp == False:
        print(save)
      final_im.save(f'out/{seed}_{save:05}.png')

def generate_mov(seed, truncation, direction_vec, scale, layers, n_frames, out_name = 'out', noise_spec = None, loop=True):
  """Generates a mov moving back and forth along the chosen direction vector"""
  # Example of reading a generated set of images, and storing as MP4.
  %mkdir out
  movieName = f'out/{out_name}.mp4'
  offset = -10
  step = 20 / n_frames
  imgs = []
  for i in log_progress(range(n_frames), name = "Generating frames"):
    print(f'\r{i} / {n_frames}', end='')
    w = model.sample_latent(1, seed=seed).cpu().numpy()

    model.truncation = truncation
    w = [w]*model.get_max_latents() # one per layer
    for l in layers:
      if l <= model.get_max_latents():
          w[l] = w[l] + direction_vec * offset * scale

    #save image and display
    out = model.sample_np(w)
    final_im = Image.fromarray((out * 255).astype(np.uint8))
    imgs.append(out)
    #increase offset
    offset += step
  if loop:
    imgs += imgs[::-1]
  with imageio.get_writer(movieName, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(img_as_ubyte(image))


seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)
start_layer = widgets.IntSlider(min=0, max=model.get_max_latents(), step=1, value=0, description='start layer: ', continuous_update=False)
end_layer = widgets.IntSlider(min=0, max=model.get_max_latents(), step=1, value=18, description='end layer: ', continuous_update=False)

# Make sure layer range is valid
def update_range_start(*args):
  end_layer.min = start_layer.value
def update_range_end(*args):
  start_layer.max = end_layer.value
start_layer.observe(update_range_start, 'value')
end_layer.observe(update_range_end, 'value')

text = widgets.Text(description="Name component here", style=style, width=200)

bot_box = widgets.HBox([seed, truncation, distance, scale, start_layer, end_layer, text])
ui = widgets.VBox([bot_box])

out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer})

display(ui, out)
text.on_submit(name_direction)

Output()

In [ ]:
print(named_directions)

{'c1': [3, 3, 10], 'c2': [3, 7, 8]}


In [ ]:
#script to generate a movie moving back and forth along the direction

direction_name = 'c3'
num_samples = 5
assert direction_name in named_directions, \
  f'"{direction_name}" not found, please save it first using the cell above.'

loc = named_directions[direction_name][0]
for i in range(num_samples):
  s = np.random.randint(0, 10000)
  generate_mov(seed = s, truncation = 0.8, direction_vec = latent_dirs[loc], scale = 2, layers=range(named_directions[direction_name][1], named_directions[direction_name][2]), n_frames = 20, out_name = f'{model_class}_{direction_name}_{i}', loop=True)

mkdir: cannot create directory ‘out’: File exists


19 / 20

mkdir: cannot create directory ‘out’: File exists


19 / 20

mkdir: cannot create directory ‘out’: File exists


19 / 20

mkdir: cannot create directory ‘out’: File exists


19 / 20

mkdir: cannot create directory ‘out’: File exists


19 / 20

In [ ]:
#@title Select from named directions

from IPython.display import display, clear_output

vardict = list(named_directions.keys())
select_variable = widgets.Dropdown(
    options=vardict,
    value=vardict[0],
    description='Select variable:',
    disabled=False,
    button_style=''
)

def set_direction(b):
    clear_output()
    random_dir = latent_dirs[named_directions[select_variable.value][0]]
    start_layer = named_directions[select_variable.value][1]
    end_layer = named_directions[select_variable.value][2]
    print(start_layer, end_layer)
    out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': fixed(start_layer), 'end': fixed(end_layer)})
    display(select_variable)
    display(ui, out)

random_dir = latent_dirs[named_directions[select_variable.value][0]]
start_layer = named_directions[select_variable.value][1]
end_layer = named_directions[select_variable.value][2]
seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)

bot_box = widgets.HBox([seed, truncation, distance, scale])
ui = widgets.VBox([bot_box])
out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': fixed(start_layer), 'end': fixed(end_layer)})

display(select_variable)
display(ui, out)

select_variable.observe(set_direction, names='value')



10 11


Dropdown(description='Select variable:', index=2, options=('c1', 'c2', 'c3'), value='c3')

Output()